In [1]:
import os
import numpy as np
import pydicom
import gdcm
from matplotlib import pyplot, cm
import plotly.plotly as py
import plotly
plotly.tools.set_credentials_file(username='arashicage', api_key='HaYe75rMnuehNkQ1cFCR')

In [2]:
directory = "/home/rodrigo/Projects/TCC/samples/PatientThreeEightSix/ct-pre-contrast-5mm-std/"
filename = "CT000024.dcm"
filepath = directory + filename

ds = pydicom.dcmread(filepath)
# pyplot.imshow(ds.pixel_array, cmap=pyplot.cm.bone)
# pyplot.imshow(ds.pixel_array) 

In [3]:
def histogram_equalization(img, no_bins):
    
    #img- the image as a numpy.array
    #the appropriate number of bins, `no_bins` in the histogram is chosen by experiments, 
    #until the contrast is convenient
    
    image_hist, bins = np.histogram(img.flatten(), no_bins, normed=True)
    csum = image_hist.cumsum() 
    cdf_mult = np.max(img) * csum / csum[-1] # cdf multiplied by a factor

    #  linear interpolation of cdf_mult to get new pixel values
    im_new = np.interp(img.flatten(), bins[:-1],  cdf_mult)

    return im_new.reshape(img.shape), cdf_mult

In [4]:
pl_bone=[[0.0, 'rgb(0, 0, 0)'],
         [0.05, 'rgb(10, 10, 14)'],
         [0.1, 'rgb(21, 21, 30)'],
         [0.15, 'rgb(33, 33, 46)'],
         [0.2, 'rgb(44, 44, 62)'],
         [0.25, 'rgb(56, 55, 77)'],
         [0.3, 'rgb(66, 66, 92)'],
         [0.35, 'rgb(77, 77, 108)'],
         [0.4, 'rgb(89, 92, 121)'],
         [0.45, 'rgb(100, 107, 132)'],
         [0.5, 'rgb(112, 123, 143)'],
         [0.55, 'rgb(122, 137, 154)'],
         [0.6, 'rgb(133, 153, 165)'],
         [0.65, 'rgb(145, 169, 177)'],
         [0.7, 'rgb(156, 184, 188)'],
         [0.75, 'rgb(168, 199, 199)'],
         [0.8, 'rgb(185, 210, 210)'],
         [0.85, 'rgb(203, 221, 221)'],
         [0.9, 'rgb(220, 233, 233)'],
         [0.95, 'rgb(238, 244, 244)'],
         [1.0, 'rgb(255, 255, 255)']]

In [5]:
def get_pl_image(dicom_filename, hist_equal=False, no_bins=None):
    #dicom_filename- a string 'filename.dcm'
    #no_bins is the number of bins for histogram when hist_equal=False, else it is None
    #returns the np.array that defines the z-value for the heatmap representing the dicom image
    
    dic_file=pydicom.read_file(dicom_filename)
    img=dic_file.pixel_array#get the image as a numpy.array
    if hist_equal and isinstance(no_bins, int):
        img_new=histogram_equalization(img, no_bins)[0]
        img_new=np.array(img_new, dtype=np.int16)
        return np.flipud(img_new)
    else:
        return np.flipud(img)

In [6]:
def DICOM_heatmap(z, title, width=600, height=600, colorscale=pl_bone):

    data=[dict(type='heatmap', 
           z=z, 
           colorscale=colorscale, 
           zsmooth='best',
           colorbar=dict(thickness=20, ticklen=4),  
              )
         ]

    axis=dict(zeroline=False, showgrid=False, ticklen=4)
    layout=dict(width=600, height=600,
            font=dict(family='Balto', size=12),
            xaxis= dict(axis),
            yaxis= dict(axis),
            title= title
            )
    return  dict(data=data, layout=layout)

In [7]:
title1="Plotly viz of a DICOM medical image<br>"
pl_img1=get_pl_image(filepath)
fig1=DICOM_heatmap(pl_img1, title1)
py.iplot(fig1, filename='exemplo')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~arashicage/0 or inside your plot.ly account where it is named 'exemplo'


/home/rodrigo/miniconda3/envs/pydicomenv/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

